# WIC

The Special Supplemental Nutrition Program for Women, Infants, and Children (WIC) provides vouchers for food and formula to low-income mothers and children deemed to be at nutritional risk.
The Food and Nutrition Service (FNS), a division of the United States Department of Agriculture (USDA), administers the WIC program.

[USDA reports](https://www.ers.usda.gov/topics/food-nutrition-assistance/wic-program/) that:
>Federal program costs for WIC totaled $4.9 billion in fiscal year 2020.

## Examples

### How earnings affect a one-person household's WIC benefit value

A pregnant woman at nutritional risk with no income is eligible for WIC, which will provide a benefit valued at \$37 per month.

In [10]:
from openfisca_us import IndividualSim
import pandas as pd
import plotly.express as px

sim = IndividualSim(year=2022)
sim.add_person(
    is_pregnant = True,
    is_wic_at_nutritional_risk = True
)

round(sim.calc("wic")[0] / 12)

37

In [37]:
sim = IndividualSim(year=2022)
sim.add_person(
    name="mother",
    employment_income=000 * 12, 
    is_mother = True,
    is_wic_at_nutritional_risk = True
)
sim.add_person(
    name="infant",
    age=0.8,
    is_wic_at_nutritional_risk = True
)

sim.add_family(name="family", members=["mother", "infant"])
sim.add_spm_unit(name="spm_unit", members=["mother", "infant"])

round(sim.calc("wic").sum() / 12)

176

What if their earnings change?
They receive the maximum benefit of \$250 per month until they reach \$720 in monthly earnings, at which point it starts phasing out until their earnings reach \$1,350.
At that point, they continue to receive the \$20 minimum monthly benefit until they reach \$2,150 per month, 200% of the poverty line, at which point they are no longer eligible.

In [38]:
sim.vary("employment_income", max=5000 * 12, step=120)

import plotly.express as px

LABELS = dict(
    employment_income="Monthly employment income",
    wic="Monthly WIC benefit value",
    mtr="Marginal tax rate",
)

df = pd.DataFrame(
    dict(
        employment_income=sim.calc("employment_income").sum(axis=0),
        wic=sim.calc("wic").sum(axis=0),
    )
)
df[["employment_income", "wic"]] = (
    df[["employment_income", "wic"]] / 12
).round()

fig = px.line(
    df,
    "employment_income",
    "wic",
    labels=LABELS,
    title="WIC benefit for a one-person household in California",
)
fig.update_layout(xaxis_tickformat="$", yaxis_tickformat="$")
fig.show()

We can also view their marginal tax rate from the program, revealing that SNAP phases out at 36 cents on the dollar through the phase-out region.
The person also experiences a cliff (infinite marginal tax rate) when they hit 200% of the poverty line.

In [3]:
fig = px.line(
    df,
    "employment_income",
    "mtr",
    labels=LABELS,
    title="SNAP marginal tax rate for a one-person household in California with $600 monthly housing costs",
)
fig.update_layout(xaxis_tickformat="$", yaxis_tickformat=".0%", yaxis_range=[0, 1])
fig.show()

### How housing costs affect a four-person household's SNAP benefits

We can also visualize how other household characteristics affect SNAP benefits.
For example, what if we vary the housing costs of a four-person household with \$2,000 monthly income?

Their SNAP benefit is \$409 per month if their housing costs are less than \$720 per month. If they spend \$1,310 per month on housing, their benefit rises 44% to \$588 per month.

In [4]:
sim = IndividualSim(year=2022)
sim.add_person(name="parent1", employment_income=1000 * 12)
sim.add_person(name="parent2", employment_income=1000 * 12)
sim.add_person(name="child1")
sim.add_person(name="child2")
sim.add_spm_unit(
    name="spm_unit",
    members=["parent1", "parent2", "child1", "child2"],
)

sim.vary("housing_cost", max=2000 * 12, step=120)

df = pd.DataFrame(
    dict(
        housing_cost=sim.calc("housing_cost")[0],
        snap=sim.calc("snap")[0],
        housing_subsidy_rate=sim.deriv("snap", "housing_cost"),
    )
)
df[["housing_cost", "snap"]] = (df[["housing_cost", "snap"]] / 12).round()


fig = px.line(
    df,
    "housing_cost",
    "snap",
    labels=LABELS,
    title="SNAP benefit for a four-person household in California with $1,000 monthly earnings",
)
fig.update_layout(
    xaxis_tickformat="$",
    yaxis_tickformat="$",
    yaxis_range=[0, df.snap.max() * 1.1],
)
fig.show()

As before, we can also see the rate at which SNAP increases with housing costs. For this household, SNAP subsidizes housing costs at a rate of 30% for housing costs ranging from \$720 to \$1,310 per month.

In [5]:
fig = px.line(
    df,
    "housing_cost",
    "housing_subsidy_rate",
    labels=LABELS,
    title="SNAP housing subsidy rate for a one-person household in California with $1,000 monthly earnings",
)
fig.update_layout(xaxis_tickformat="$", yaxis_tickformat=".0%")
fig.show()